In [ ]:
import mne
import os
import glob

# Load ALL resting state files from raw directory
raw_dir = os.path.join('..', '..', 'raw')

# Find all files with "RestingState" in the name
resting_files = glob.glob(os.path.join(raw_dir, "*RestingState*.set"))

print(f"Found {len(resting_files)} resting state files:")
for f in resting_files:
    print(f"  - {os.path.basename(f)}")

# Load all files
raw_list = []
for file_path in resting_files:
    print(f"\nLoading {os.path.basename(file_path)}...")
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    raw_list.append(raw)
    print(f"  ✓ Channels: {len(raw.ch_names)}, Duration: {raw.times[-1]:.1f}s")

print(f"\n✓ Loaded {len(raw_list)} resting state recordings")

In [ ]:
# low pass filter
raw = [r.filter(l_freq=1, h_freq=45, fir_design='firwin') for r in raw_list]

In [ ]:
from asrpy import ASR

for r in raw:
    asr = ASR(sfreq=r.info['sfreq'], cutoff=15)
    asr.fit(r)

In [ ]:
from scipy import signal as sp_signal
import matplotlib.pyplot as plt
import numpy as np

# Get data
data = raw[0].get_data()

# Create LARGE figure
fig, ax = plt.subplots(figsize=(18, 8))  # Super wide and tall


# Compute and plot PSD for all channels
for i, ch_name in enumerate(raw[0].ch_names):
    freqs, psd = sp_signal.welch(data[i], fs=raw[0].info['sfreq'], nperseg=2048)
    
    # Only plot frequencies 0-60 Hz with values > 0
    mask = (freqs >= 0) & (freqs <= 60) & (psd > 0)
    ax.semilogy(freqs[mask], psd[mask], alpha=0.3, linewidth=0.5)

# Make it readable
ax.set_xlabel('Frequency (Hz)', fontsize=14)
ax.set_ylabel('Power Spectral Density (V²/Hz)', fontsize=14)
ax.set_title('PSD - All Channels (0-60 Hz)', fontsize=16, fontweight='bold')
ax.set_xlim(0, 60)
ax.grid(True, alpha=0.3)

# Add brain wave band markers
ax.axvspan(0.5, 4, alpha=0.1, color='purple', label='Delta (0.5-4 Hz)')
ax.axvspan(4, 8, alpha=0.1, color='blue', label='Theta (4-8 Hz)')
ax.axvspan(8, 13, alpha=0.1, color='green', label='Alpha (8-13 Hz)')
ax.axvspan(13, 30, alpha=0.1, color='orange', label='Beta (13-30 Hz)')
ax.axvspan(30, 50, alpha=0.1, color='red', label='Gamma (30-50 Hz)')

ax.legend(fontsize=12, loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Save cleaned data in different formats

# Ensure the processed directory exists to avoid FileNotFoundError
processed_dir = os.path.join('..', '..', 'processed')
os.makedirs(processed_dir, exist_ok=True)

# Option 1: Save as .fif (MNE's native format - RECOMMENDED)
for i, r in enumerate(raw):
    output_path_fif = os.path.join(processed_dir, f'resting_state_cleaned_{i}.fif')
    r.save(output_path_fif, overwrite=True)
    print(f"✓ Saved as FIF: {output_path_fif}")

# Also save channel names and sampling frequency
for i, r in enumerate(raw):

    metadata = {
        'ch_names': r.ch_names,
        'sfreq': r.info['sfreq'],
        'n_channels': len(r.ch_names),
        'n_times': r.n_times
    }
    metadata_path = os.path.join(processed_dir, f'resting_state_metadata{i}.npy')
    np.save(metadata_path, metadata)
    print(f"✓ Saved metadata: {metadata_path}")

print("\n" + "="*60)
print("SAVED CLEANED DATA")
print("="*60)
print(f"Channels: {len(raw[0].ch_names)}")
print(f"Sampling rate: {raw[0].info['sfreq']} Hz")
print(f"Duration: {raw[0].times[-1]:.1f} seconds")
print(f"Data shape: {raw[0].get_data().shape}")
print("="*60)